<a href="https://colab.research.google.com/github/binh1206/PINN-SciANN/blob/main/DampedHarmonicOscillator/Forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.10 keras==2.10 sciann
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()
!conda config --add channels conda-forge
!conda config --add channels nvidia
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sciann as sn
from sciann.utils.math import diff, sign, sin

In [ ]:
w0 = 20
sigma = 2 # under-damped: sigma < w0
w = np.sqrt(w0**2 - sigma**2)
phi = np.arctan(-(0+ sigma)/w)
A = 1/(2*np.cos(phi))
def func_x(t):
 return np.exp(-sigma*t) *2*A*np.cos(w*t+phi )
t_data = np.linspace (0 ,1 ,15) # 15 cap (t,x)
x_data = func_x(t_data )

In [ ]:
t = sn.Variable ('t') # dau vao
x = sn.Field ('x') # dau ra
x = sn.Functional (
    fields =[x],
    variables =[t],
    hidden_layers =8*[20] , # mang than kinh 8*20
    activation ='sin') # ham kich hoat

In [ ]:
x_t = diff(x, t)
x_tt = diff(x, t, order =2)
ode = 1e-4*(x_tt + 2* sigma *x_t + (w0 **2) *x) # ODE
# dieu kien ban dau
TOL = 1e-6;
x0 = (1 - sign(t - TOL))*(x -1)
v0 = (1 - sign(t - TOL))*(x_t -0)

In [ ]:
t_train = np.linspace (0 ,3 ,100) # mien thoi gian huan luyen
m = sn.SciModel([t], [sn.PDE( ode), x0 , v0], loss_func ='mse', optimizer='adam')
h = m.train(
    [ t_train ],
     [0, 0, 0],
    epochs =10000,
    adaptive_weights =100,
    learning_rate =1e-3,
    batch_size =50,
    verbose =1)
plt.semilogy(h.history['loss']) # ve do thi Loss vs Epoch
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
t_plot = np.linspace(0,3,5000)
x_plot = func_x(t_plot)
plt.plot (t_plot, x_plot, label='Exact solution ')
t_pred = np.linspace(0,3,100) # mien thoi gian cho du doan
x_pred = x.eval([t_pred]) # x du doan
plt.plot(t_pred, x_pred , label='SciANN prediction ')
plt.xlabel ('$t ~[s]$')
plt.ylabel ('$x ~[m]$')
plt.legend ()
plt.show ()